# Evacuacion en Mina Subterranea
## BFS vs UCS vs Greedy vs A*
**Taller IA 2026**

In [ ]:
from collections import deque
import heapq, json
from IPython.display import HTML, display

COSTS = {'.': 1, 'S': 1, 'E': 1, 'F': 1, 'G': 10, '#': float('inf'), 'X': float('inf')}

class Mina:
    def __init__(self, txt):
        self.g = [list(l) for l in txt.strip().split('\n')]
        self.h, self.w = len(self.g), len(self.g[0])
    def vecinos(self, p):
        r, c = p
        return [(r+dr, c+dc) for dr, dc in [(-1,0),(1,0),(0,-1),(0,1)] if 0 <= r+dr < self.h and 0 <= c+dc < self.w and self.g[r+dr][c+dc] not in '#X']
    def costo(self, p): return COSTS.get(self.g[p[0]][p[1]], 1)
    def encontrar(self, chars): return [(r,c) for r in range(self.h) for c in range(self.w) if self.g[r][c] in chars]
    def clonar(self): m = Mina.__new__(Mina); m.g, m.h, m.w = [r[:] for r in self.g], self.h, self.w; return m

def reconstruir(cf, ini, fin):
    p, c = [], fin
    while c: p.append(c); c = cf.get(c)
    return p[::-1]

def bfs(m, ini, metas):
    q, cf, exp = deque([ini]), {ini: None}, 0
    while q:
        c = q.popleft(); exp += 1
        if c in metas: return reconstruir(cf, ini, c), sum(m.costo(p) for p in reconstruir(cf, ini, c)[1:]), exp
        for v in m.vecinos(c):
            if v not in cf: q.append(v); cf[v] = c
    return [], float('inf'), exp

def ucs(m, ini, metas):
    q, cf, cs, exp, n = [(0,0,ini)], {ini: None}, {ini: 0}, 0, 0
    while q:
        _, _, c = heapq.heappop(q); exp += 1
        if c in metas: return reconstruir(cf, ini, c), cs[c], exp
        for v in m.vecinos(c):
            nc = cs[c] + m.costo(v)
            if v not in cs or nc < cs[v]: cs[v] = nc; n += 1; heapq.heappush(q, (nc, n, v)); cf[v] = c
    return [], float('inf'), exp

def h(p, metas): return min(abs(p[0]-g[0]) + abs(p[1]-g[1]) for g in metas) if metas else 0

def greedy(m, ini, metas):
    q, cf, vis, exp, n = [(h(ini,metas),0,ini)], {ini: None}, set(), 0, 0
    while q:
        _, _, c = heapq.heappop(q)
        if c in vis: continue
        vis.add(c); exp += 1
        if c in metas: return reconstruir(cf, ini, c), sum(m.costo(p) for p in reconstruir(cf, ini, c)[1:]), exp
        for v in m.vecinos(c):
            if v not in vis: n += 1; heapq.heappush(q, (h(v,metas), n, v)); cf.setdefault(v, c)
    return [], float('inf'), exp

def astar(m, ini, metas):
    q, cf, cs, exp, n = [(h(ini,metas),0,ini)], {ini: None}, {ini: 0}, 0, 0
    while q:
        _, _, c = heapq.heappop(q); exp += 1
        if c in metas: return reconstruir(cf, ini, c), cs[c], exp
        for v in m.vecinos(c):
            nc = cs[c] + m.costo(v)
            if v not in cs or nc < cs[v]: cs[v] = nc; n += 1; heapq.heappush(q, (nc + h(v,metas), n, v)); cf[v] = c
    return [], float('inf'), exp

ALGOS = {'BFS': bfs, 'UCS': ucs, 'Greedy': greedy, 'A*': astar}
print('Algoritmos cargados')

In [ ]:
MAPA_A = '''\n####################\n#S................E#\n#.####.####.####...#\n#......#.......#...#\n#.####.#.#####.#...#\n#......#.......#...#\n#.####.#.#####.#...#\n#......#..S....#...#\n#.####.#.#####.#...#\n#......#...........#\n#..S...#.......#...#\n#......#.......#..F#\n####################\n'''

MAPA_B = '''\n####################\n#S..GGGGGGGGGGGGGGE#\n#.##GG####.####....#\n#...GGG#.......#...#\n#.####.#.#####.#...#\n#......#.......#...#\n#.####.#.#####.#...#\n#......#..S....#...#\n#.####.#.#####.#...#\n#......#...........#\n#..S...#.......#...#\n#......#.......#..F#\n####################\n'''

MAPA_C = '''\n####################\n#S................E#\n#.####.####.####...#\n#......#.......#...#\n#.####.#.#####.#...#\n#......#.......#...#\n#.####.#.#####.#...#\n#......#..S....#...#\n#.####.#.#####.#...#\n#......#...........#\n#..S...#.......#...#\n#......#.......#..F#\n####################\n'''

ESCENARIOS = {'A': ('Normal', 'Sin incidentes', MAPA_A, []), 'B': ('Gas', 'Gas en ruta', MAPA_B, []), 'C': ('Derrumbe', 'Derrumbe paso 2', MAPA_C, [(2,(1,8)),(2,(1,9)),(2,(1,10))])}
print('Escenarios cargados')

In [ ]:
def simular(esc_id, algo):
    nombre, desc, mapa, eventos = ESCENARIOS[esc_id]
    m = Mina(mapa)
    starts, metas = m.encontrar('S')[:3], m.encontrar('EF')
    rutas = {}
    for i, s in enumerate(starts):
        ruta, _, _ = ALGOS[algo](m, s, metas)
        rutas[i] = ruta if ruta else [s]
    path, costo, nodos = ALGOS[algo](m.clonar(), starts[0], metas)
    pos = {i: list(starts[i]) for i in range(len(starts))}
    idx = {i: 0 for i in range(len(starts))}
    llegaron = {i: False for i in range(len(starts))}
    pasos = [{'paso': 0, 'pos': {f'W{i}': pos[i][:] for i in pos}, 'mapa': [''.join(r) for r in m.g], 'evento': None}]
    replans = 0
    for t in range(1, 50):
        ev = None
        for et, ep in eventos:
            if et == t: m.g[ep[0]][ep[1]] = 'X'; ev = 'DERRUMBE'
        if ev:
            replans += 1
            for i in range(len(starts)):
                if not llegaron[i]:
                    ruta, _, _ = ALGOS[algo](m, tuple(pos[i]), metas)
                    rutas[i] = ruta if ruta else [tuple(pos[i])]; idx[i] = 0
        for i in range(len(starts)):
            if llegaron[i]: continue
            if rutas[i] and idx[i] < len(rutas[i]) - 1: idx[i] += 1; pos[i] = list(rutas[i][idx[i]])
            if tuple(pos[i]) in metas: llegaron[i] = True
        pasos.append({'paso': t, 'pos': {f'W{i}': pos[i][:] for i in pos}, 'mapa': [''.join(r) for r in m.g], 'evento': ev})
        if all(llegaron.values()): break
    return {'esc': f'Escenario {esc_id}', 'algo': algo, 'path': [list(p) for p in path], 'costo': costo, 'nodos': nodos, 'replans': replans, 'pasos': pasos}
print('Simulador cargado')

In [ ]:
def generar_html(res):
    data = {f"{r['esc']}_{r['algo']}": r for r in res}
    filas = ''.join([f'<tr class="row" data-k="{r["esc"]}_{r["algo"]}"><td>{r["esc"]}</td><td><b>{r["algo"]}</b></td><td>{len(r["path"])-1}</td><td>{r["costo"]:.0f}</td><td>{r["nodos"]}</td><td>{r["replans"]}</td></tr>' for r in res])
    data_json = json.dumps(data)
    return '''<div id="app"><style>#app{font-family:Arial;background:#1a1a2e;color:#fff;padding:20px;border-radius:12px}#app .panel{background:rgba(255,255,255,0.1);border-radius:12px;padding:20px;margin:15px 0}#app h1,#app h2{color:#3498db}#app table{width:100%;border-collapse:collapse}#app th,#app td{padding:10px;text-align:center;border-bottom:1px solid rgba(255,255,255,0.1)}#app th{background:rgba(52,152,219,0.3)}#app .row{cursor:pointer}#app .row:hover,#app .row.sel{background:rgba(52,152,219,0.3)}#app #map{display:grid;gap:2px;justify-content:center;margin:15px 0}#app .cell{width:28px;height:28px;display:flex;align-items:center;justify-content:center;border-radius:3px;font-size:12px}#app .c-wall{background:#2c3e50}#app .c-floor{background:#ecf0f1;color:#333}#app .c-start{background:#3498db}#app .c-exit{background:#27ae60}#app .c-refuge{background:#16a085}#app .c-gas{background:#e74c3c}#app .c-collapse{background:#8b4513}#app .c-path{background:#f1c40f!important}#app .c-worker{background:#e67e22!important;border-radius:50%}#app .controls{display:flex;gap:10px;justify-content:center;margin:15px 0}#app button{padding:8px 16px;border:none;border-radius:6px;cursor:pointer;background:#3498db;color:#fff}#app .legend{display:flex;gap:12px;justify-content:center;flex-wrap:wrap;margin:10px 0}#app .leg{display:flex;align-items:center;gap:5px}#app .leg-c{width:20px;height:20px;border-radius:3px}#app .info{background:rgba(0,0,0,0.2);padding:12px;border-radius:8px;margin:10px 0}#app .info-row{display:flex;justify-content:space-between;padding:4px 0}#app .alert{background:#e74c3c;padding:10px;border-radius:8px;text-align:center;margin:10px 0;display:none}#app .reas{display:grid;grid-template-columns:repeat(4,1fr);gap:10px;margin:10px 0}#app .reas-item{background:rgba(0,0,0,0.2);padding:12px;border-radius:8px;text-align:center}#app .reas-item h4{font-size:1.8em;color:#3498db}</style><div class="panel"><h1>Evacuacion en Mina</h1><p style="text-align:center;color:#aaa">BFS vs UCS vs Greedy vs A*</p></div><div class="panel"><h2>Modelo REAS</h2><div class="reas"><div class="reas-item"><h4>R</h4><b>Reactividad</b></div><div class="reas-item"><h4>E</h4><b>Entorno</b></div><div class="reas-item"><h4>A</h4><b>Autonomia</b></div><div class="reas-item"><h4>S</h4><b>Sociabilidad</b></div></div></div><div class="panel"><h2>Resultados</h2><table><tr><th>Escenario</th><th>Algoritmo</th><th>Pasos</th><th>Costo</th><th>Nodos</th><th>Replans</th></tr>''' + filas + '''</table></div><div class="panel"><h2>Visualizacion</h2><div class="controls"><button onclick="prev()">Ant</button><button id="playBtn" onclick="toggle()">Play</button><button onclick="next()">Sig</button><span id="stepTxt">Paso: 0/0</span><button onclick="reset()">Reset</button></div><div class="legend"><div class="leg"><div class="leg-c" style="background:#2c3e50"></div>Pared</div><div class="leg"><div class="leg-c" style="background:#ecf0f1"></div>Galeria</div><div class="leg"><div class="leg-c" style="background:#27ae60"></div>Salida</div><div class="leg"><div class="leg-c" style="background:#16a085"></div>Refugio</div><div class="leg"><div class="leg-c" style="background:#e74c3c"></div>Gas</div><div class="leg"><div class="leg-c" style="background:#8b4513"></div>Derrumbe</div><div class="leg"><div class="leg-c" style="background:#f1c40f"></div>Ruta</div><div class="leg"><div class="leg-c" style="background:#e67e22;border-radius:50%"></div>Trabajador</div></div><div id="map"></div><div id="alert" class="alert"></div><div class="info"><div class="info-row"><span>Escenario:</span><span id="iScen">-</span></div><div class="info-row"><span>Algoritmo:</span><span id="iAlgo">-</span></div><div class="info-row"><span>Costo:</span><span id="iCost">-</span></div><div class="info-row"><span>Nodos:</span><span id="iNodes">-</span></div></div></div><script>const DATA=''' + data_json + ''';let cur=null,step=0,playing=false,timer=null;document.querySelectorAll("#app .row").forEach(r=>r.onclick=()=>select(r));document.querySelector("#app .row")?.click();function select(row){document.querySelectorAll("#app .row").forEach(r=>r.classList.remove("sel"));row.classList.add("sel");cur=DATA[row.dataset.k];step=0;stop();document.getElementById("iScen").textContent=cur.esc;document.getElementById("iAlgo").textContent=cur.algo;document.getElementById("iCost").textContent=cur.costo.toFixed(1);document.getElementById("iNodes").textContent=cur.nodos;render();}function render(){if(!cur)return;const s=cur.pasos[step],map=document.getElementById("map"),lines=s.mapa;map.style.gridTemplateColumns="repeat("+lines[0].length+",28px)";map.innerHTML="";for(let r=0;r<lines.length;r++)for(let c=0;c<lines[0].length;c++){const ch=lines[r][c],cell=document.createElement("div");const cls={"#":"c-wall",".":"c-floor","S":"c-start","E":"c-exit","F":"c-refuge","G":"c-gas","X":"c-collapse"};cell.className="cell "+(cls[ch]||"c-floor");if(ch==="G")cell.textContent="X";if(ch==="E")cell.textContent="E";if(ch==="F")cell.textContent="F";if(ch==="X")cell.textContent="!";if(cur.path.some(p=>p[0]===r&&p[1]===c)&&"EFG".indexOf(ch)<0)cell.classList.add("c-path");for(const k in s.pos)if(s.pos[k][0]===r&&s.pos[k][1]===c){cell.classList.add("c-worker");cell.textContent="W";}map.appendChild(cell);}document.getElementById("stepTxt").textContent="Paso: "+step+"/"+(cur.pasos.length-1);const alert=document.getElementById("alert");if(s.evento){alert.textContent=s.evento;alert.style.display="block";}else alert.style.display="none";}function next(){if(cur&&step<cur.pasos.length-1){step++;render();}else stop();}function prev(){if(cur&&step>0){step--;render();}}function toggle(){playing?stop():start();}function start(){playing=true;document.getElementById("playBtn").textContent="Pause";timer=setInterval(next,400);}function stop(){playing=false;document.getElementById("playBtn").textContent="Play";clearInterval(timer);}function reset(){step=0;stop();render();}</script></div>'''
print('HTML cargado')

In [ ]:
print('EVACUACION MINA')
res = []
for e in ['A', 'B', 'C']:
    print(f'Escenario {e}: {ESCENARIOS[e][1]}')
    for a in ['BFS', 'UCS', 'Greedy', 'A*']:
        r = simular(e, a)
        res.append(r)
        print(f'   {a:7} | Pasos:{len(r["path"])-1:3} | Costo:{r["costo"]:5.0f} | Nodos:{r["nodos"]:4}')
    print()
display(HTML(generar_html(res)))

# Evacuacion en Mina Subterranea
## Comparacion de Algoritmos: BFS vs UCS vs Greedy vs A*

**Taller de Introduccion a la IA - 2026**

In [ ]:
from collections import deque
import heapq, json
from IPython.display import HTML, display

COSTS = {'.': 1, 'S': 1, 'E': 1, 'F': 1, 'G': 10, '#': float('inf'), 'X': float('inf')}

class Mina:
    def __init__(self, txt):
        self.g = [list(l) for l in txt.strip().split('\n')]
        self.h, self.w = len(self.g), len(self.g[0])
    
    def vecinos(self, p):
        r, c = p
        return [(r+dr, c+dc) for dr, dc in [(-1,0),(1,0),(0,-1),(0,1)]
                if 0 <= r+dr < self.h and 0 <= c+dc < self.w and self.g[r+dr][c+dc] not in '#X']
    
    def costo(self, p): return COSTS.get(self.g[p[0]][p[1]], 1)
    def encontrar(self, chars): return [(r,c) for r in range(self.h) for c in range(self.w) if self.g[r][c] in chars]
    def clonar(self): m = Mina.__new__(Mina); m.g, m.h, m.w = [r[:] for r in self.g], self.h, self.w; return m

def reconstruir(cf, ini, fin):
    p, c = [], fin
    while c: p.append(c); c = cf.get(c)
    return p[::-1]

def bfs(m, ini, metas):
    q, cf, exp = deque([ini]), {ini: None}, 0
    while q:
        c = q.popleft(); exp += 1
        if c in metas: return reconstruir(cf, ini, c), sum(m.costo(p) for p in reconstruir(cf, ini, c)[1:]), exp
        for v in m.vecinos(c):
            if v not in cf: q.append(v); cf[v] = c
    return [], float('inf'), exp

def ucs(m, ini, metas):
    q, cf, cs, exp, n = [(0,0,ini)], {ini: None}, {ini: 0}, 0, 0
    while q:
        _, _, c = heapq.heappop(q); exp += 1
        if c in metas: return reconstruir(cf, ini, c), cs[c], exp
        for v in m.vecinos(c):
            nc = cs[c] + m.costo(v)
            if v not in cs or nc < cs[v]: cs[v] = nc; n += 1; heapq.heappush(q, (nc, n, v)); cf[v] = c
    return [], float('inf'), exp

def h(p, metas): return min(abs(p[0]-g[0]) + abs(p[1]-g[1]) for g in metas) if metas else 0

def greedy(m, ini, metas):
    q, cf, vis, exp, n = [(h(ini,metas),0,ini)], {ini: None}, set(), 0, 0
    while q:
        _, _, c = heapq.heappop(q)
        if c in vis: continue
        vis.add(c); exp += 1
        if c in metas: return reconstruir(cf, ini, c), sum(m.costo(p) for p in reconstruir(cf, ini, c)[1:]), exp
        for v in m.vecinos(c):
            if v not in vis: n += 1; heapq.heappush(q, (h(v,metas), n, v)); cf.setdefault(v, c)
    return [], float('inf'), exp

def astar(m, ini, metas):
    q, cf, cs, exp, n = [(h(ini,metas),0,ini)], {ini: None}, {ini: 0}, 0, 0
    while q:
        _, _, c = heapq.heappop(q); exp += 1
        if c in metas: return reconstruir(cf, ini, c), cs[c], exp
        for v in m.vecinos(c):
            nc = cs[c] + m.costo(v)
            if v not in cs or nc < cs[v]: cs[v] = nc; n += 1; heapq.heappush(q, (nc + h(v,metas), n, v)); cf[v] = c
    return [], float('inf'), exp

ALGOS = {'BFS': bfs, 'UCS': ucs, 'Greedy': greedy, 'A*': astar}
print('Algoritmos cargados')

In [ ]:
MAPA_A = '''
####################
#S................E#
#.####.####.####...#
#......#.......#...#
#.####.#.#####.#...#
#......#.......#...#
#.####.#.#####.#...#
#......#..S....#...#
#.####.#.#####.#...#
#......#...........#
#..S...#.......#...#
#......#.......#..F#
####################
'''

MAPA_B = '''
####################
#S..GGGGGGGGGGGGGGE#
#.##GG####.####....#
#...GGG#.......#...#
#.####.#.#####.#...#
#......#.......#...#
#.####.#.#####.#...#
#......#..S....#...#
#.####.#.#####.#...#
#......#...........#
#..S...#.......#...#
#......#.......#..F#
####################
'''

MAPA_C = '''
####################
#S................E#
#.####.####.####...#
#......#.......#...#
#.####.#.#####.#...#
#......#.......#...#
#.####.#.#####.#...#
#......#..S....#...#
#.####.#.#####.#...#
#......#...........#
#..S...#.......#...#
#......#.......#..F#
####################
'''

ESCENARIOS = {
    'A': ('Normal', 'Sin incidentes', MAPA_A, []),
    'B': ('Gas', 'Gas en ruta directa', MAPA_B, []),
    'C': ('Derrumbe', 'Derrumbe en paso 2', MAPA_C, [(2,(1,8)),(2,(1,9)),(2,(1,10))]),
}
print('Escenarios cargados')

In [ ]:
def simular(esc_id, algo):
    nombre, desc, mapa, eventos = ESCENARIOS[esc_id]
    m = Mina(mapa)
    starts, metas = m.encontrar('S')[:3], m.encontrar('EF')
    
    rutas = {}
    for i, s in enumerate(starts):
        ruta, _, _ = ALGOS[algo](m, s, metas)
        rutas[i] = ruta if ruta else [s]
    
    path, costo, nodos = ALGOS[algo](m.clonar(), starts[0], metas)
    
    pos = {i: list(starts[i]) for i in range(len(starts))}
    idx = {i: 0 for i in range(len(starts))}
    llegaron = {i: False for i in range(len(starts))}
    pasos = [{'paso': 0, 'pos': {f'W{i}': pos[i][:] for i in pos}, 'mapa': [''.join(r) for r in m.g], 'evento': None}]
    replans = 0
    
    for t in range(1, 50):
        ev = None
        for et, ep in eventos:
            if et == t:
                m.g[ep[0]][ep[1]] = 'X'
                ev = 'DERRUMBE'
        
        if ev:
            replans += 1
            for i in range(len(starts)):
                if not llegaron[i]:
                    ruta, _, _ = ALGOS[algo](m, tuple(pos[i]), metas)
                    rutas[i] = ruta if ruta else [tuple(pos[i])]
                    idx[i] = 0
        
        for i in range(len(starts)):
            if llegaron[i]: continue
            if rutas[i] and idx[i] < len(rutas[i]) - 1:
                idx[i] += 1
                pos[i] = list(rutas[i][idx[i]])
            if tuple(pos[i]) in metas: llegaron[i] = True
        
        pasos.append({'paso': t, 'pos': {f'W{i}': pos[i][:] for i in pos}, 'mapa': [''.join(r) for r in m.g], 'evento': ev})
        if all(llegaron.values()): break
    
    return {'esc': f'Escenario {esc_id}', 'algo': algo, 'path': [list(p) for p in path],
            'costo': costo, 'nodos': nodos, 'replans': replans, 'pasos': pasos}

print('Funcion simular cargada')

In [ ]:
def generar_html(res):
    data = {f"{r['esc']}_{r['algo']}": r for r in res}
    filas = ''
    for r in res:
        filas += f'<tr class="row" data-k="{r["esc"]}_{r["algo"]}"><td>{r["esc"]}</td><td><b>{r["algo"]}</b></td><td>{len(r["path"])-1}</td><td>{r["costo"]:.0f}</td><td>{r["nodos"]}</td><td>{r["replans"]}</td></tr>'
    
    data_json = json.dumps(data, ensure_ascii=False)
    
    html = '''<div id="app">
<style>
#app{font-family:Arial;background:#1a1a2e;color:#fff;padding:20px;border-radius:12px}
#app .panel{background:rgba(255,255,255,0.1);border-radius:12px;padding:20px;margin:15px 0}
#app h1,#app h2{color:#3498db}
#app table{width:100%;border-collapse:collapse}
#app th,#app td{padding:10px;text-align:center;border-bottom:1px solid rgba(255,255,255,0.1)}
#app th{background:rgba(52,152,219,0.3)}
#app .row{cursor:pointer}#app .row:hover,#app .row.sel{background:rgba(52,152,219,0.3)}
#app #map{display:grid;gap:2px;justify-content:center;margin:15px 0}
#app .cell{width:28px;height:28px;display:flex;align-items:center;justify-content:center;border-radius:3px;font-size:12px}
#app .c-wall{background:#2c3e50}#app .c-floor{background:#ecf0f1;color:#333}#app .c-start{background:#3498db}
#app .c-exit{background:#27ae60}#app .c-refuge{background:#16a085}#app .c-gas{background:#e74c3c}#app .c-collapse{background:#8b4513}
#app .c-path{background:#f1c40f!important}#app .c-worker{background:#e67e22!important;border-radius:50%}
#app .controls{display:flex;gap:10px;justify-content:center;margin:15px 0}
#app button{padding:8px 16px;border:none;border-radius:6px;cursor:pointer;background:#3498db;color:#fff}
#app .legend{display:flex;gap:12px;justify-content:center;flex-wrap:wrap;margin:10px 0}
#app .leg{display:flex;align-items:center;gap:5px}#app .leg-c{width:20px;height:20px;border-radius:3px}
#app .info{background:rgba(0,0,0,0.2);padding:12px;border-radius:8px;margin:10px 0}
#app .info-row{display:flex;justify-content:space-between;padding:4px 0}
#app .alert{background:#e74c3c;padding:10px;border-radius:8px;text-align:center;margin:10px 0;display:none}
#app .reas{display:grid;grid-template-columns:repeat(4,1fr);gap:10px;margin:10px 0}
#app .reas-item{background:rgba(0,0,0,0.2);padding:12px;border-radius:8px;text-align:center}
#app .reas-item h4{font-size:1.8em;color:#3498db}
</style>

<div class="panel"><h1>Evacuacion en Mina Subterranea</h1>
<p style="text-align:center;color:#aaa">Comparacion: BFS vs UCS vs Greedy vs A*</p></div>

<div class="panel"><h2>Modelo REAS</h2>
<div class="reas">
<div class="reas-item"><h4>R</h4><b>Reactividad</b><br><small>Reacciona a gas/derrumbes</small></div>
<div class="reas-item"><h4>E</h4><b>Entorno</b><br><small>Mapa dinamico discreto</small></div>
<div class="reas-item"><h4>A</h4><b>Autonomia</b><br><small>Decide ruta sin humanos</small></div>
<div class="reas-item"><h4>S</h4><b>Sociabilidad</b><br><small>Agentes cooperando</small></div>
</div></div>

<div class="panel"><h2>Resultados</h2>
<p style="margin-bottom:10px">Clic en una fila para ver animacion</p>
<table><tr><th>Escenario</th><th>Algoritmo</th><th>Pasos</th><th>Costo</th><th>Nodos</th><th>Replans</th></tr>
''' + filas + '''</table></div>

<div class="panel"><h2>Visualizacion</h2>
<div class="controls">
<button onclick="prev()">Anterior</button><button id="playBtn" onclick="toggle()">Play</button>
<button onclick="next()">Siguiente</button><span id="stepTxt">Paso: 0/0</span>
<button onclick="reset()">Reset</button></div>
<div class="legend">
<div class="leg"><div class="leg-c" style="background:#2c3e50"></div>Pared</div>
<div class="leg"><div class="leg-c" style="background:#ecf0f1"></div>Galeria</div>
<div class="leg"><div class="leg-c" style="background:#27ae60"></div>Salida</div>
<div class="leg"><div class="leg-c" style="background:#16a085"></div>Refugio</div>
<div class="leg"><div class="leg-c" style="background:#e74c3c"></div>Gas</div>
<div class="leg"><div class="leg-c" style="background:#8b4513"></div>Derrumbe</div>
<div class="leg"><div class="leg-c" style="background:#f1c40f"></div>Ruta</div>
<div class="leg"><div class="leg-c" style="background:#e67e22;border-radius:50%"></div>Trabajador</div>
</div>
<div id="map"></div>
<div id="alert" class="alert"></div>
<div class="info">
<div class="info-row"><span>Escenario:</span><span id="iScen">-</span></div>
<div class="info-row"><span>Algoritmo:</span><span id="iAlgo">-</span></div>
<div class="info-row"><span>Costo:</span><span id="iCost">-</span></div>
<div class="info-row"><span>Nodos:</span><span id="iNodes">-</span></div>
</div></div>

<div class="panel"><h2>Algoritmos</h2>
<div class="reas">
<div class="reas-item"><b>BFS</b><br><small>Menos pasos, ignora costos</small></div>
<div class="reas-item"><b>UCS</b><br><small>Menor costo total</small></div>
<div class="reas-item"><b>Greedy</b><br><small>Rapido, no optimo</small></div>
<div class="reas-item"><b>A*</b><br><small>Optimo y eficiente</small></div>
</div></div>

<script>
const DATA=''' + data_json + ''';
let cur=null,step=0,playing=false,timer=null;

document.querySelectorAll("#app .row").forEach(r=>r.onclick=()=>select(r));
document.querySelector("#app .row")?.click();

function select(row){
  document.querySelectorAll("#app .row").forEach(r=>r.classList.remove("sel"));
  row.classList.add("sel");
  cur=DATA[row.dataset.k];step=0;stop();
  document.getElementById("iScen").textContent=cur.esc;
  document.getElementById("iAlgo").textContent=cur.algo;
  document.getElementById("iCost").textContent=cur.costo.toFixed(1);
  document.getElementById("iNodes").textContent=cur.nodos;
  render();
}

function render(){
  if(!cur)return;
  const s=cur.pasos[step],map=document.getElementById("map"),lines=s.mapa;
  map.style.gridTemplateColumns="repeat("+lines[0].length+",28px)";
  map.innerHTML="";
  for(let r=0;r<lines.length;r++)for(let c=0;c<lines[0].length;c++){
    const ch=lines[r][c],cell=document.createElement("div");
    const cls={"#":"c-wall",".":"c-floor","S":"c-start","E":"c-exit","F":"c-refuge","G":"c-gas","X":"c-collapse"};
    cell.className="cell "+(cls[ch]||"c-floor");
    if(ch==="G")cell.textContent="X";
    if(ch==="E")cell.textContent="E";
    if(ch==="F")cell.textContent="F";
    if(ch==="X")cell.textContent="!";
    if(cur.path.some(p=>p[0]===r&&p[1]===c)&&"EFG".indexOf(ch)<0)cell.classList.add("c-path");
    for(const k in s.pos)if(s.pos[k][0]===r&&s.pos[k][1]===c){cell.classList.add("c-worker");cell.textContent="W";}
    map.appendChild(cell);
  }
  document.getElementById("stepTxt").textContent="Paso: "+step+"/"+(cur.pasos.length-1);
  const alert=document.getElementById("alert");
  if(s.evento){alert.textContent=s.evento;alert.style.display="block";}else alert.style.display="none";
}

function next(){if(cur&&step<cur.pasos.length-1){step++;render();}else stop();}
function prev(){if(cur&&step>0){step--;render();}}
function toggle(){playing?stop():start();}
function start(){playing=true;document.getElementById("playBtn").textContent="Pause";timer=setInterval(next,400);}
function stop(){playing=false;document.getElementById("playBtn").textContent="Play";clearInterval(timer);}
function reset(){step=0;stop();render();}
</script>
</div>'''
    
    return html

print('Generador HTML cargado')

In [ ]:
print('EVACUACION MINA - BFS/UCS/Greedy/A*')
print()
res = []
for e in ['A', 'B', 'C']:
    print(f'Escenario {e}: {ESCENARIOS[e][1]}')
    for a in ['BFS', 'UCS', 'Greedy', 'A*']:
        r = simular(e, a)
        res.append(r)
        print(f'   {a:7} | Pasos:{len(r["path"])-1:3} | Costo:{r["costo"]:5.0f} | Nodos:{r["nodos"]:4}')
    print()

print('Simulacion completada!')
display(HTML(generar_html(res)))

# ⛏️ Evacuación en Mina Subterránea
## Comparación de Algoritmos: BFS vs UCS vs Greedy vs A*

**Taller de Introducción a la IA - 2026**

Este notebook implementa un agente inteligente para evacuación de trabajadores en una mina subterránea, comparando 4 algoritmos de búsqueda.

### Modelo REAS
- **R**eactividad: Reacciona a gas y derrumbes
- **E**ntorno: Mapa dinámico discreto
- **A**utonomía: Decide ruta sin intervención humana
- **S**ociabilidad: Múltiples agentes cooperando

## 1. Imports y Configuración

In [ ]:
from collections import deque
import heapq, json
from IPython.display import HTML, display

COSTS = {'.': 1, 'S': 1, 'E': 1, 'F': 1, 'G': 10, '#': float('inf'), 'X': float('inf')}
print("✅ Configuración cargada")

## 2. Clase Mina (Representación del Entorno)

In [ ]:
class Mina:
    def __init__(self, txt):
        self.g = [list(l) for l in txt.strip().split('\n')]
        self.h, self.w = len(self.g), len(self.g[0])
    
    def vecinos(self, p):
        r, c = p
        return [(r+dr, c+dc) for dr, dc in [(-1,0),(1,0),(0,-1),(0,1)]
                if 0 <= r+dr < self.h and 0 <= c+dc < self.w and self.g[r+dr][c+dc] not in '#X']
    
    def costo(self, p): return COSTS.get(self.g[p[0]][p[1]], 1)
    def encontrar(self, chars): return [(r,c) for r in range(self.h) for c in range(self.w) if self.g[r][c] in chars]
    def clonar(self): m = Mina.__new__(Mina); m.g, m.h, m.w = [r[:] for r in self.g], self.h, self.w; return m

print("✅ Clase Mina definida")

## 3. Algoritmos de Búsqueda

In [ ]:
def reconstruir(cf, ini, fin):
    p, c = [], fin
    while c: p.append(c); c = cf.get(c)
    return p[::-1]

def bfs(m, ini, metas):
    """Breadth-First Search - Encuentra camino más corto en pasos"""
    q, cf, exp = deque([ini]), {ini: None}, 0
    while q:
        c = q.popleft(); exp += 1
        if c in metas: return reconstruir(cf, ini, c), sum(m.costo(p) for p in reconstruir(cf, ini, c)[1:]), exp
        for v in m.vecinos(c):
            if v not in cf: q.append(v); cf[v] = c
    return [], float('inf'), exp

def ucs(m, ini, metas):
    """Uniform Cost Search - Encuentra camino de menor costo"""
    q, cf, cs, exp, n = [(0,0,ini)], {ini: None}, {ini: 0}, 0, 0
    while q:
        _, _, c = heapq.heappop(q); exp += 1
        if c in metas: return reconstruir(cf, ini, c), cs[c], exp
        for v in m.vecinos(c):
            nc = cs[c] + m.costo(v)
            if v not in cs or nc < cs[v]: cs[v] = nc; n += 1; heapq.heappush(q, (nc, n, v)); cf[v] = c
    return [], float('inf'), exp

def h(p, metas): 
    """Heurística Manhattan"""
    return min(abs(p[0]-g[0]) + abs(p[1]-g[1]) for g in metas) if metas else 0

def greedy(m, ini, metas):
    """Greedy Best-First Search - Sigue la heurística"""
    q, cf, vis, exp, n = [(h(ini,metas),0,ini)], {ini: None}, set(), 0, 0
    while q:
        _, _, c = heapq.heappop(q)
        if c in vis: continue
        vis.add(c); exp += 1
        if c in metas: return reconstruir(cf, ini, c), sum(m.costo(p) for p in reconstruir(cf, ini, c)[1:]), exp
        for v in m.vecinos(c):
            if v not in vis: n += 1; heapq.heappush(q, (h(v,metas), n, v)); cf.setdefault(v, c)
    return [], float('inf'), exp

def astar(m, ini, metas):
    """A* - Óptimo y eficiente"""
    q, cf, cs, exp, n = [(h(ini,metas),0,ini)], {ini: None}, {ini: 0}, 0, 0
    while q:
        _, _, c = heapq.heappop(q); exp += 1
        if c in metas: return reconstruir(cf, ini, c), cs[c], exp
        for v in m.vecinos(c):
            nc = cs[c] + m.costo(v)
            if v not in cs or nc < cs[v]: cs[v] = nc; n += 1; heapq.heappush(q, (nc + h(v,metas), n, v)); cf[v] = c
    return [], float('inf'), exp

ALGOS = {"BFS": bfs, "UCS": ucs, "Greedy": greedy, "A*": astar}
print("✅ Algoritmos definidos: BFS, UCS, Greedy, A*")

## 4. Mapas y Escenarios

In [ ]:
MAPA_A = """
####################
#S................E#
#.####.####.####...#
#......#.......#...#
#.####.#.#####.#...#
#......#.......#...#
#.####.#.#####.#...#
#......#..S....#...#
#.####.#.#####.#...#
#......#...........#
#..S...#.......#...#
#......#.......#..F#
####################
"""

MAPA_B = """
####################
#S..GGGGGGGGGGGGGGE#
#.##GG####.####....#
#...GGG#.......#...#
#.####.#.#####.#...#
#......#.......#...#
#.####.#.#####.#...#
#......#..S....#...#
#.####.#.#####.#...#
#......#...........#
#..S...#.......#...#
#......#.......#..F#
####################
"""

MAPA_C = """
####################
#S................E#
#.####.####.####...#
#......#.......#...#
#.####.#.#####.#...#
#......#.......#...#
#.####.#.#####.#...#
#......#..S....#...#
#.####.#.#####.#...#
#......#...........#
#..S...#.......#...#
#......#.......#..F#
####################
"""

ESCENARIOS = {
    "A": ("Normal", "Sin incidentes", MAPA_A, []),
    "B": ("Gas", "Gas en ruta directa", MAPA_B, []),
    "C": ("Derrumbe", "Derrumbe en paso 2", MAPA_C, [(2,(1,8)),(2,(1,9)),(2,(1,10))]),
}

print("✅ Escenarios definidos:")
for k, v in ESCENARIOS.items():
    print(f"   {k}: {v[0]} - {v[1]}")

## 5. Función de Simulación

In [ ]:
def simular(esc_id, algo):
    nombre, desc, mapa, eventos = ESCENARIOS[esc_id]
    m = Mina(mapa)
    starts, metas = m.encontrar('S')[:3], m.encontrar('EF')
    
    rutas = {}
    for i, s in enumerate(starts):
        ruta, _, _ = ALGOS[algo](m, s, metas)
        rutas[i] = ruta if ruta else [s]
    
    path, costo, nodos = ALGOS[algo](m.clonar(), starts[0], metas)
    
    pos = {i: list(starts[i]) for i in range(len(starts))}
    idx = {i: 0 for i in range(len(starts))}
    llegaron = {i: False for i in range(len(starts))}
    pasos = [{"paso": 0, "pos": {f"W{i}": pos[i][:] for i in pos}, "mapa": [''.join(r) for r in m.g], "evento": None}]
    replans = 0
    
    for t in range(1, 50):
        ev = None
        for et, ep in eventos:
            if et == t:
                m.g[ep[0]][ep[1]] = 'X'
                ev = f"⚠️ DERRUMBE en ({ep[0]},{ep[1]})"
        
        if ev:
            replans += 1
            for i in range(len(starts)):
                if not llegaron[i]:
                    ruta, _, _ = ALGOS[algo](m, tuple(pos[i]), metas)
                    rutas[i] = ruta if ruta else [tuple(pos[i])]
                    idx[i] = 0
        
        for i in range(len(starts)):
            if llegaron[i]: continue
            if rutas[i] and idx[i] < len(rutas[i]) - 1:
                idx[i] += 1
                pos[i] = list(rutas[i][idx[i]])
            if tuple(pos[i]) in metas: llegaron[i] = True
        
        pasos.append({"paso": t, "pos": {f"W{i}": pos[i][:] for i in pos}, "mapa": [''.join(r) for r in m.g], "evento": ev})
        if all(llegaron.values()): break
    
    return {"esc": f"Escenario {esc_id}", "algo": algo, "path": [list(p) for p in path],
            "costo": costo, "nodos": nodos, "replans": replans, "pasos": pasos}

print("✅ Función de simulación definida")

## 6. Generador de Visualización HTML

In [ ]:
def generar_html(res):
    data = {f"{r['esc']}_{r['algo']}": r for r in res}
    filas = "\n".join(f'''<tr class="row" data-k="{r['esc']}_{r['algo']}">
        <td>{r['esc']}</td><td><b>{r['algo']}</b></td>
        <td>{len(r['path'])-1}</td><td>{r['costo']:.0f}</td>
        <td>{r['nodos']}</td><td>{r['replans']}</td></tr>''' for r in res)
    
    html = f'''<div id="evacuation-app">
<style>
#evacuation-app *{{margin:0;padding:0;box-sizing:border-box}}
#evacuation-app {{font-family:Arial;background:linear-gradient(135deg,#1a1a2e,#16213e);color:#fff;padding:20px;border-radius:12px}}
#evacuation-app .panel{{background:rgba(255,255,255,0.1);border-radius:12px;padding:20px;margin:15px 0}}
#evacuation-app h1{{text-align:center;margin-bottom:20px}}
#evacuation-app h2{{color:#3498db;border-bottom:2px solid #3498db;padding-bottom:8px;margin-bottom:15px}}
#evacuation-app table{{width:100%;border-collapse:collapse}}
#evacuation-app th,#evacuation-app td{{padding:10px;text-align:center;border-bottom:1px solid rgba(255,255,255,0.1)}}
#evacuation-app th{{background:rgba(52,152,219,0.3)}}
#evacuation-app .row{{cursor:pointer}}#evacuation-app .row:hover,#evacuation-app .row.sel{{background:rgba(52,152,219,0.3)}}
#evacuation-app #map{{display:grid;gap:2px;justify-content:center;margin:15px 0}}
#evacuation-app .cell{{width:28px;height:28px;display:flex;align-items:center;justify-content:center;border-radius:3px;font-size:12px}}
#evacuation-app .c-wall{{background:#2c3e50}}#evacuation-app .c-floor{{background:#ecf0f1;color:#333}}#evacuation-app .c-start{{background:#3498db}}
#evacuation-app .c-exit{{background:#27ae60}}#evacuation-app .c-refuge{{background:#16a085}}#evacuation-app .c-gas{{background:#e74c3c}}#evacuation-app .c-collapse{{background:#8b4513}}
#evacuation-app .c-path{{background:#f1c40f!important;color:#333}}#evacuation-app .c-worker{{background:#e67e22!important;border-radius:50%}}
#evacuation-app .controls{{display:flex;gap:10px;justify-content:center;flex-wrap:wrap;margin:15px 0}}
#evacuation-app button{{padding:8px 16px;border:none;border-radius:6px;cursor:pointer;background:#3498db;color:#fff}}
#evacuation-app button:hover{{background:#2980b9}}
#evacuation-app .legend{{display:flex;gap:12px;justify-content:center;flex-wrap:wrap;margin:10px 0}}
#evacuation-app .leg{{display:flex;align-items:center;gap:5px}}#evacuation-app .leg-c{{width:20px;height:20px;border-radius:3px}}
#evacuation-app .info{{background:rgba(0,0,0,0.2);padding:12px;border-radius:8px;margin:10px 0}}
#evacuation-app .info-row{{display:flex;justify-content:space-between;padding:4px 0}}
#evacuation-app .alert{{background:#e74c3c;padding:10px;border-radius:8px;text-align:center;margin:10px 0;display:none}}
#evacuation-app .reas{{display:grid;grid-template-columns:repeat(auto-fit,minmax(150px,1fr));gap:10px;margin:10px 0}}
#evacuation-app .reas-item{{background:rgba(0,0,0,0.2);padding:12px;border-radius:8px;text-align:center}}
#evacuation-app .reas-item h4{{font-size:1.8em;color:#3498db}}
</style>

<div class="panel"><h1>⛏️ Evacuación en Mina Subterránea</h1>
<p style="text-align:center;color:#aaa">Comparación: BFS vs UCS vs Greedy vs A*</p></div>

<div class="panel"><h2>🤖 Modelo REAS</h2>
<div class="reas">
<div class="reas-item"><h4>R</h4><b>Reactividad</b><br><small>Reacciona a gas/derrumbes</small></div>
<div class="reas-item"><h4>E</h4><b>Entorno</b><br><small>Mapa dinámico discreto</small></div>
<div class="reas-item"><h4>A</h4><b>Autonomía</b><br><small>Decide ruta sin humanos</small></div>
<div class="reas-item"><h4>S</h4><b>Sociabilidad</b><br><small>Agentes cooperando</small></div>
</div></div>

<div class="panel"><h2>📊 Resultados</h2>
<p style="margin-bottom:10px">Clic en una fila para ver animación</p>
<table><tr><th>Escenario</th><th>Algoritmo</th><th>Pasos</th><th>Costo</th><th>Nodos</th><th>Replans</th></tr>
{filas}</table></div>

<div class="panel"><h2>🎮 Visualización</h2>
<div class="controls">
<button onclick="prev()">⏮ Ant</button><button id="playBtn" onclick="toggle()">▶ Play</button>
<button onclick="next()">Sig ⏭</button><span id="stepTxt">Paso: 0/0</span>
<button onclick="reset()">🔄</button></div>
<div class="legend">
<div class="leg"><div class="leg-c" style="background:#2c3e50"></div>Pared</div>
<div class="leg"><div class="leg-c" style="background:#ecf0f1"></div>Galería</div>
<div class="leg"><div class="leg-c" style="background:#27ae60"></div>Salida</div>
<div class="leg"><div class="leg-c" style="background:#16a085"></div>Refugio</div>
<div class="leg"><div class="leg-c" style="background:#e74c3c"></div>Gas</div>
<div class="leg"><div class="leg-c" style="background:#8b4513"></div>Derrumbe</div>
<div class="leg"><div class="leg-c" style="background:#f1c40f"></div>Ruta</div>
<div class="leg"><div class="leg-c" style="background:#e67e22;border-radius:50%"></div>Trabajador</div>
</div>
<div id="map"></div>
<div id="alert" class="alert"></div>
<div class="info">
<div class="info-row"><span>Escenario:</span><span id="iScen">-</span></div>
<div class="info-row"><span>Algoritmo:</span><span id="iAlgo">-</span></div>
<div class="info-row"><span>Costo:</span><span id="iCost">-</span></div>
<div class="info-row"><span>Nodos:</span><span id="iNodes">-</span></div>
</div></div>

<div class="panel"><h2>📚 Algoritmos</h2>
<div class="reas">
<div class="reas-item"><b>BFS</b><br><small>Menos pasos, ignora costos</small></div>
<div class="reas-item"><b>UCS</b><br><small>Menor costo total</small></div>
<div class="reas-item"><b>Greedy</b><br><small>Rápido, no óptimo</small></div>
<div class="reas-item"><b>A*</b><br><small>Óptimo y eficiente</small></div>
</div></div>

<script>
const DATA=DATA_PLACEHOLDER;
let cur=null,step=0,playing=false,timer=null;

document.querySelectorAll('#evacuation-app .row').forEach(r=>r.onclick=()=>select(r));
document.querySelector('#evacuation-app .row')?.click();

function select(row){{
  document.querySelectorAll('#evacuation-app .row').forEach(r=>r.classList.remove('sel'));
  row.classList.add('sel');
  cur=DATA[row.dataset.k];step=0;stop();
  document.getElementById('iScen').textContent=cur.esc;
  document.getElementById('iAlgo').textContent=cur.algo;
  document.getElementById('iCost').textContent=cur.costo.toFixed(1);
  document.getElementById('iNodes').textContent=cur.nodos;
  render();
}}

function render(){{
  if(!cur)return;
  const s=cur.pasos[step],map=document.getElementById('map'),lines=s.mapa;
  map.style.gridTemplateColumns=`repeat(${{lines[0].length}},28px)`;
  map.innerHTML='';
  for(let r=0;r<lines.length;r++)for(let c=0;c<lines[0].length;c++){{
    const ch=lines[r][c],cell=document.createElement('div');
    cell.className='cell '+({{ '#':'c-wall','.':'c-floor','S':'c-start','E':'c-exit','F':'c-refuge','G':'c-gas','X':'c-collapse'}}[ch]||'c-floor');
    if(ch==='G')cell.textContent='☠️';
    if(ch==='E')cell.textContent='🚪';
    if(ch==='F')cell.textContent='🏠';
    if(ch==='X')cell.textContent='💥';
    if(cur.path.some(p=>p[0]===r&&p[1]===c)&&!'EFG'.includes(ch))cell.classList.add('c-path');
    for(const[,pos]of Object.entries(s.pos))if(pos[0]===r&&pos[1]===c){{cell.classList.add('c-worker');cell.textContent='👷';}}
    map.appendChild(cell);
  }}
  document.getElementById('stepTxt').textContent=`Paso: ${{step}}/${{cur.pasos.length-1}}`;
  const alert=document.getElementById('alert');
  if(s.evento){{alert.textContent=s.evento;alert.style.display='block';}}else alert.style.display='none';
}}

function next(){{if(cur&&step<cur.pasos.length-1){{step++;render();}}else stop();}}
function prev(){{if(cur&&step>0){{step--;render();}}}}
function toggle(){{playing?stop():start();}}
function start(){{playing=true;document.getElementById('playBtn').textContent='⏸';timer=setInterval(next,400);}}
function stop(){{playing=false;document.getElementById('playBtn').textContent='▶ Play';clearInterval(timer);}}
function reset(){{step=0;stop();render();}}
</script>
</div>'''.replace('DATA_PLACEHOLDER', json.dumps(data, ensure_ascii=False))
    
    return html

print("✅ Generador HTML definido")

## 7. 🚀 Ejecutar Simulación

In [ ]:
print("⛏️  EVACUACIÓN MINA - BFS/UCS/Greedy/A*\n")
res = []
for e in ["A", "B", "C"]:
    print(f"📍 Escenario {e}: {ESCENARIOS[e][1]}")
    for a in ["BFS", "UCS", "Greedy", "A*"]:
        r = simular(e, a)
        res.append(r)
        print(f"   {a:7} | Pasos:{len(r['path'])-1:3} | Costo:{r['costo']:5.0f} | Nodos:{r['nodos']:4}")
    print()

print("\n✅ Simulación completada! Visualización abajo:")
display(HTML(generar_html(res)))